<a href="https://colab.research.google.com/github/GuillermoSego/TSF/blob/main/CovCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# El doble apuntador

Cuando necesitamos modificar un vector en una función, podemos hacerlo de dos maneras, sobreescribiendo el vector (haciendo una copia) que es **por valor**, o escribiendo sobre el vector original que es **por referencia**. En esta última opción ahorramos tiempo de ejecución al ahorrar memoria. Para hacer esto en una función que no es de tipo *void* necesitamos utilizar un apuntador doble. El apuntador doble es un apuntador que guarda la dirección de otro apuntador. Entonces para modificar el valor original solo hay que cambiar la dirección de la dirección.

Para un apuntador utilizizamos la notación del doble 
```{C}

#include <stdio.h>

int main(){

  // Declaramos un vector
  double Vector[12];
  
  // Apuntamos al vector
  double* PtrV = Vector;

  // Apuntamos al apuntador del vector
  double **PtrPtr = &PtrV;

  return 0;
}

```

Haciendo esto podemos modificar los elementos por referencia en una función void.

El siguiente código es un ejemplo de esto

In [ ]:
%%writefile doble.c

#include <stdio.h>

// Esta rutina llena un arreglo con numeros del 0 al 9

// Le pasamos un apuntador doble
double Full(double** ptrd)
{
    int i = 0;
    
    do {
        
        // Con el doble ** llenamos el vector original por referencia
        **ptrd = i ;
        
        /*
        Con cualquiera de estas dos instrucciones recorremos el apuntador 
        del vector original
        */

        //*ptrd = *ptrd+1;
        ++ *ptrd;
        
        i++; 
        
    } while (i<10);
    
    return i;
}

int main()
{
    
    // Declaramos los apuntadores
    double vector[10];
    double* ptrvec = vector;
    double** ptr2 = &ptrvec;
    
    int j; 
    
    /*
    Llamamos la funcion 
    La igualamos a un valor entero ya que la funcion no es de tipo void
    */
    j = Full(ptr2);
    
    int i = 0;
    
    printf("Imprimimos el vector\n");

    do {
        
        printf("%lf\n", vector[i]);
        
         i++;
         
    } while (i<10);

    printf("El numero final de iteraciones: %d", j);
    
    return 0;
}

Overwriting doble.c


In [ ]:
%%shell

gcc doble.c -o doble
./doble

Imprimimos el vector
0.000000
1.000000
2.000000
3.000000
4.000000
5.000000
6.000000
7.000000
8.000000
9.000000
El numero final de iteraciones: 10

## CUIDADO
El programa anterior es mucha complicación. Utilizamos los apuntadores dobles no para modificar el contenido de la memoria (para esto tenemos los apuntadores normales), sino **para modificar la memoria**, ie. cuando usamos memoria dinámica y necesitamos ampliar la memoria. 



# Programa de cálculo de la matriz de covarianza, a partir de un CSV.

In [171]:
%%writefile Cov.c

#include <stdio.h>

#define BF 100

// Manejador de errores
void ErrorManagement(int e)
{
    if (e == 0) {
        
        printf("Error al abrir el archivo\n");
    }

}

// Esta rutina calcula el producto punto entre dos vectores
double DotProd(double x[], double y[], int n)
{
    double sum = *(x++) * *(y ++ );
    --n ;
    
    do {
        sum += *( x++ ) * *(y ++ );
    } while (--n);
    
    return sum;
}

/*
Esta rutina calcula la varianza de los grupos de datos y llena la diagonal 
principal de la matriz de covarianza
*/
void Variance(double* sum_xPtr, double* CovPtr, double* DatPtr[], int N, int M )
{
    // En estas variables se van a hacer las sumas
    double sum_x = 0, sum_x2 = 0;  
    
    int i = 0, k = 0;
    
    do {  
    
        sum_x = 0, sum_x2 = 0;    
    
            do {
                
                // Hacemos las sumas de x y x^2
                sum_x2 += *(DatPtr[k]) * *(DatPtr[k]);
                sum_x += *(DatPtr[k] ++);
                
                i ++;
            
            } while(i<M);
    
        // Guardamos el valor de sum x en un arreglo
        *sum_xPtr = sum_x;
        sum_xPtr ++;
        
        /*
        Calculamos la varianza y directamente guardamos estos valores en la 
        matriz de covarianza
        */
        *CovPtr = (M * (sum_x2) - (sum_x) * (sum_x)) / ( M*(M-1) );
        
        // Recorremos el apuntador para tener solo los valores de la diagonal
        CovPtr  = CovPtr + (N+1);
        
        k ++;
        i = 0;
    
    } while(k < N);
    
    
}

/*
En esta rutina se calcula la covarianza entre todas las variables y se guarda 
en la matriz de covarianza
*/
void Covariance(double* sum_xPtr, double* CovPtr, double* DatPtr[], int N, int M )
{
    
    int i = 0, k = 0;
    
    do {
        
        do {
            
            /*
            Calculamos la covarianza y lo guardamos directamente en la matriz. 
            La covarianza se calcula por renglon, un ciclo es para los renglones 
            y otro para las columnas. Avanzamos de manera que permutamos en 
            cada renglon todas las variables, permutando una variable fija con 
            todas las demas. De esta manera, si en el primer renglon tenemos x,
            hacemo xy, xz, xw, xe, xd, ... Cuando avanzamos al siguiente renglon
            hacemos yz, yw, ye, yd, ... Avanzamos zw, ze, zd, ... y asi hasta 
            acabar
            */
            *CovPtr = (M * DotProd(DatPtr[i],  DatPtr[k+1], M) - 
            (*sum_xPtr)* *(sum_xPtr + (k+1)) ) / ( M*(M-1) );
            
            //printf("%lf\n", DotProd(DatPtr[i],  DatPtr[k+1], M) );
            //printf("%d\n", k);
            
            //*CovPtr = 45;
            
            /*
            Llenamos los lugares transpuestos de la matriz sumando los lugares 
            que corresponden. (k+1)(n-1)
            */
            *(CovPtr + (k+1)*(N-1)) = *CovPtr;
            
            CovPtr ++;
            
            k ++;    
        
        // Movemos los indices para estar en la diagonal superior de la matriz
        
         } while( k < (N - 1) - i );
        
        // Apuntamos en la dirección superior de la diagonal principal 
        CovPtr = CovPtr + (i+2);
        sum_xPtr ++;
        
        k = 0;
        i ++;
    } while( i < N-1);

    
}

// Esta rutina muestra una matriz de mxn
void MatrixShow(int m, int n, double v[])
{
    
     
    int i, j, k=0;
    
    for(i=0 ; i<m ; i++){
        
        for (j=0 ; j<n ; j ++){
            
            printf("%lf \t", v[k]);
            k ++;
        }
        
        printf("\n");
    }
    
    
    
}

// Esta rutina calcula la matriz transpuesta de un vector y la guarda en otro
void MatrixT(int n, int m, double v[], double* VecPtr )
{

  double* vPtr = v;

  int i = 0, j = 0;
 
  do {

    do {
        
       *VecPtr = *vPtr;

        VecPtr ++; 

        vPtr += m;

      j ++;
    } while (j<n);   

    vPtr = v + (i+1);

    j = 0;
    i ++;
  } while (i<m);


}

int* OpenCSV(char* arg, double DatV[BF])
{
    
  // Declaramos una variable tipo FILE
  FILE* Doc;

  // Abrimos el archivo. "r": Abrir un archivo existente para leer. 
  // Hacemos que el apuntador Doc apunte en la direccion de los datos
  Doc = fopen ( arg, "r" );
 
  // Manejamos el error: no se pueda abrir el archivo
  if (Doc == NULL)
  {
      ErrorManagement( 0 );
      return 0;
  } 
 
  // Continuamos con la estructura para leer los datos
 
  // Primero creamos un buffer para guardar los datos
	char buffer[BF];

  // Se lee la primera linea y se almacena en el buffer como variable char
	fgets(buffer, BF, Doc);
  
  /* 
 
    En este punto todo esta almacenado en el buffer como una sola cadena de 
    caracteres. Por lo que para poder acceso a los datos se tiene que convertir
    esta cadena a binario, separando por comas "," ya que un archivo tipo csv
    separa los datos por comas.
 
  */
  
  // Creamos un apuntador que apunte a la cadena de caracteres
  char* BfPtr = buffer;
 
  // Calculamos el número de columnas contando las comas de la primera linea
  int column = 0;
    
  do{
      column += ( *BfPtr == ',' );

  } while( *BfPtr ++);
 
  // Convertimos a binario
 
  // Apuntamos en la direccion del buffer
  BfPtr = buffer;
 
  // Apuntador al vector de datos
  double* DatPtr = DatV;
  
  // Creamos un contador igual al número de columnas
  // Declaramos el parámetro adicional para la función sscanf
  int i = column, length;
 
  // Leemos el primer renglon 

  do{
        
      BfPtr += length;
      sscanf( BfPtr, "%lf,%n", DatPtr ++, &length );
        
    } while(i--);

  /*
  
    En este punto en el vector DatV tenemos guardado la primer fila de los datos.
    Ahora tenemos que guardar las siguientes filas.
 
    El apuntador del buffer BfPtr se encuentra apuntando hacia el primer dato 
    de la última columna, para asegurarnos de que apuntamos exactamente al ultimo
    elemento de la primera fila, le sumamos la longitud length

  */
  BfPtr += length;

  /*
 
  Leemos las siguientes lineas del buffer y las ponemos en el apuntador 
  de los datos

  */
  sscanf( BfPtr, "%lf", DatPtr); 

  // Recorremos el apuntador de los datos
  i = column;
  DatPtr = DatV;

	do {
		*DatPtr ++;
	} while( i-- );
		
	// Se termina de leer el archivo
	
  /*
 
  En este momento leemos los demas datos del apuntador Doc, y los guardamos en
  el apuntador de los datos DatPtr, sin embargo, tenemos que llegar al final de
  los datos. Esto lo hacemos creando una variable llamada dummy en la que guar-
  damos el momento en el que se llega al final, es decir cuando tenemos que los 
  datos tienen la forma "%lf%c"

 */
  
  int Error = 0, col = column+1; 
  char dummy;

	do{

		Error = fscanf(Doc, "%lf%c", DatPtr ++, &dummy);
         
	} while(Error > 0);


  // Cerramos el archivo, es muy importante cerrar el archivo
  fclose ( Doc );
 
  static int size[2];
  size[0] = (DatPtr - DatV)/ (col);
  size[1] = col;
 
  return size;

}

int main(int nArg, char *arg[])
{

    /*
 
      Declaramos un vector en el que vamos a guardar los datos.
      Le apuntamos dos veces.
 
    */
    double VecM[BF]; 
 
    // Creamos un vector en el que vamos a guardar los valores de N y M
    int size[2];
    int* sizePtr = size;

    // Abrimos el archivo y guardamos en el vector DatM los datos
    sizePtr = OpenCSV(*(arg+1), VecM);
 
    int SA = *sizePtr, ST = *(sizePtr+1);
 
    // Declaramos un nuevo vector en el que guardaremos la transpuesta 
    double DatM[ST*SA];
    double* DatTPtr = DatM;
    
    // Determinamos la transpuesta
    MatrixT(SA, ST, VecM, DatTPtr);


    // Declaramos la matriz de la Covarianza y le apuntamos
    double CovM[ST*ST];
    double* CovPtr = CovM;
    
    /*
    Hacemos un apuntador de apuntadores para manejar los datos
    El unico problema del programa es que para ejecutar el programa hay que 
    reservar la memoria segun el número de columnas
    */
    double* DatPtr[] = {DatM, DatM + SA, DatM + 2*SA, DatM + 3*SA, DatM + 4*SA};

    // Declaramos un arreglo para las sumas y le apuntamos     
    double sum_xM[ST];
    double* sum_xPtr = sum_xM;

    // Varianza
    Variance(sum_xPtr, CovPtr, DatPtr, ST, SA);
    
    // Nos cercioramos que estamos apuntando en la dirección correcta
    DatPtr[ST - 1] = &DatM[0];
    
    CovPtr = CovM + 1;
    sum_xPtr = sum_xM;


    // Covarianza 
    Covariance(sum_xPtr, CovPtr, DatPtr, ST, SA);
    
    // Se imprime la matriz
    printf( "La matriz de Covarianza es: \n");
    MatrixShow(ST,ST, CovM);
    
    
    return 0;
}


Overwriting Cov.c


In [176]:
%%shell

gcc Cov.c -o output
./output Datos.csv

La matriz de Covarianza es: 
0.798571 	0.734286 	0.679286 	0.734286 	0.679286 	
0.734286 	0.734286 	0.734286 	0.679286 	0.734286 	
0.679286 	0.734286 	0.734286 	0.679286 	0.798571 	
0.734286 	0.679286 	0.679286 	0.798571 	0.734286 	
0.679286 	0.734286 	0.798571 	0.734286 	0.734286 	
